In [1]:
import spacy
from spacy.matcher import Matcher
import re
import datetime as dt

## Using the Matcher object from SpaCy

In [2]:
nlp = spacy.load("nb_core_news_lg")

In [3]:
matcher = Matcher(nlp.vocab)
pattern = [{"LOWER": "marius"}, {"LOWER": "dioli"}]

In [4]:
tx = "Hei, mitt navn er Marius Dioli. Eg er nysgjerrig på kor mange poeng eg har."

In [5]:
def on_match(matcher, doc, id, matches):
      print('Matched!', matches)

In [6]:
matcher.add("MD matcher", on_match, pattern)

In [7]:
#matcher = Matcher(nlp.vocab)

In [8]:
doc = nlp(tx)

In [9]:
matches = matcher(doc)

Matched! [(10180913855843715433, 5, 7)]


In [10]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

10180913855843715433 MD matcher 5 7 Marius Dioli


In [11]:
reg =  [{"TEXT": {"REGEX": r'(\b\d{11}\b)|(\b\d{6}\s\d{5}\b)'}}]


In [12]:
tx="Mitt navn er robindra og mitt fødselsnummer er 15044216652"

In [13]:
def checksum(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # The matched span
    fmt = "%d%m%y"
    fnr = span[0].text
    date = fnr[:6]
    prsn = fnr[-5:]
    try:
        int(date)
    except:
        return
    try:
        dt.datetime.strptime(date, fmt)
    except ValueError:
        print("Invalid date")
    d1 = int(date[0])
    d2 = int(date[1])

    m1 = int(date[2])
    m2 = int(date[3])

    y1 = int(date[4])
    y2 = int(date[5])

    i1 = int(prsn[0])
    i2 = int(prsn[1])
    i3 = int(prsn[2])

    k1 = int(prsn[3])
    k2 = int(prsn[4])
    k1_mod = (3 * d1 + 7 * d2 + 6 * m1 + m2 + 8 * y1 + 9 * y2 + 4 * i1 + 5 * i2 + 2 * i3) % 11
    new_k1 = 0 if k1_mod == 0 else (11 - k1_mod)

    if k1 != new_k1:
        print("Invalid personnummer")

    k2_mod = (5 * d1 + 4 * d2 + 3 * m1 + 2 * m2 + 7 * y1 + 6 * y2 + 5 * i1 + 4 * i2 + 3 * i3 + 2 * k1) % 11
    new_k2 = 0 if k2_mod == 0 else (11 - k2_mod)

    if k2 != new_k2:
        print("Invalid personnummer")

In [14]:
matcher.add("fnr RegEx_med_checksum", checksum, reg)
doc = nlp(tx)
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, start, end, span.text)

1737304374787103023 8 9 15044216652


## Using the Entity Ruler object from SpaCy

In [15]:
from spacy.lang.nb import Norwegian
from spacy.pipeline import EntityRuler

In [16]:
patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]

In [17]:
reg = [{"label": "FNR", "pattern":[{"TEXT": {"REGEX": r'(\b\d{11}\b)|(\b\d{6}\s\d{5}\b)'}}]}]

In [18]:
tx="Jeg er Robindra og mitt fødselsnummer er 15044216652"

In [19]:
nlp = spacy.load("nb_core_news_lg")
#nlp = Norwegian() is an empty model
ruler = EntityRuler(nlp)
ruler.add_patterns(reg)
nlp.add_pipe(ruler)
from collections import defaultdict

In [20]:
#Example text
txt = "Selv om det hører inn under sakprosaen, er det mange forfattere som skriver essayistisk; som altså vil si assosiativt og sirkulært i formen. Mange vil kalle essayet en blandingsgenre; Mia Bull Gundersen skriver for eksempel: «Essayet er en utpreget litterær blandingsform og befinner seg et sted mellom fiksjon og sakprosa» (1). En del tekster kan være essayistiske i formen, selv om de faller inn under andre genrebetegnelser. Dette gjelder både artikler, kronikker, petiter, kåserier og romaner. Essayet er en forholdsvis kort tekst, skrevet i førsteperson entall i en ikke-konkluderende form. Jo Bech-Karlsen (2) sier essayet skal være kunnskapsrikt, men ikke lærd i teoretisk vitenskapelig forstand. Ofte stiller essayet flere spørsmål enn det besvarer, og forfatteren av essayet henvender seg gjerne til sin likemann; en kunnskapsrik og opplyst leser. I motsetning til en artikkel, som er bygget opp med en innledning, drøfting og konklusjon, har essayet verken innledning eller avslutning. Leseren får ikke nødvendigvis svar på de spørsmålene teksten stiller, men skal likevel være klokere enn før han eller hun gikk i gang med lesningen. I likhet med essayet selv, omfatter essayets historie flere tradisjoner og avstikkere. De fleste betrakter Michel de Montaigne som essayets far. Montaigne (1533–1592) var en fransk adelsmann, politiker, filosof og forfatter som hadde stor innflytelse i sin samtid. Som trettiåring trakk han seg tilbake til familiens slott i Bordeaux, for i ensomhet å reflektere og skrive, i sitt tårn, omgitt av sine bøker. Begrepet essay er hentet fra tittelen på hans bøker: Les Essais I–III, som kom ut mellom 1780 og 1788. Det franske begrepet kan oversettes med forsøk, som peker tilbake på essayets åpne, prøvende og ikke-konkluderende form. Selv fortsatte Montaigne å redigere og endre på essayene sine resten av livet. Dette forsterker inntrykket av essayet som uferdig og åpent. Det er ofte store og evige spørsmål som drøftes i essayet, samtidig som ingenting synes å være for smått."

In [21]:
doc = nlp(txt)
print([(ent.text, ent.label_, str(ent.start), str(ent.end)) for ent in doc.ents])

[('Mia Bull Gundersen', 'PER', '33', '36'), ('Essayet', 'PER', '41', '42'), ('Essayet', 'PER', '92', '93'), ('Bech-Karlsen', 'ORG', '109', '110'), ('Michel de Montaigne', 'PER', '222', '225'), ('Montaigne', 'PER', '229', '230'), ('Bordeaux', 'LOC', '261', '262'), ('Montaigne', 'PER', '323', '324')]


In [22]:
beams = nlp.entity.beam_parse([doc], beam_width=16, beam_density=0.0001)
for score, ents in nlp.entity.moves.get_beam_parses(beams[0]):
    print (score, ents)
    entity_scores = defaultdict(float)
    for start, end, label in ents:
        # print ("here")
        entity_scores[(start, end, label)] += score
        print ('entity_scores', entity_scores)

for (start, end, label),value in entity_scores.items():
        if label == 'LOCATION':
            print (start, tokens[start], value)

1.0 [(33, 36, 'PER'), (41, 42, 'PER'), (92, 93, 'PER'), (109, 110, 'ORG'), (222, 225, 'PER'), (229, 230, 'PER'), (261, 262, 'LOC'), (323, 324, 'PER')]
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0, (109, 110, 'ORG'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0, (109, 110, 'ORG'): 1.0, (222, 225, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 1.0, (109, 110, 'ORG'): 1.0, (222, 225, 'PER'): 1.0, (229, 230, 'PER'): 1.0})
entity_scores defaultdict(<class 'float'>, {(33, 36, 'PER'): 1.0, (41, 42, 'PER'): 1.0, (92, 93, 'PER'): 

### Testing custom package

In [23]:
from nav_pii_anon.spacy.spacy_model import SpacyModel

In [24]:
nlp = SpacyModel(spacy.load("nb_core_news_lg"))

In [25]:
nlp.predict(tx)

[('Robindra', 'PER', 2, 3)]


In [26]:
nlp.add_patterns()

In [27]:
nlp.predict(tx)

[('Robindra', 'PER', 2, 3), ('15044216652', 'FNR', 7, 8)]
